<a href="https://colab.research.google.com/github/MrudulMascarenhas/INS_LAB/blob/main/hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math

In [2]:
def mod_inv(a, m):
    """Compute the modular inverse of a modulo m using Extended Euclidean Algorithm."""
    def egcd(a, b):
        if a == 0:
            return (b, 0, 1)
        else:
            g, y, x = egcd(b % a, a)
            return (g, x - (b // a) * y, y)
    g, x, _ = egcd(a, m)
    if g != 1:
        raise Exception("Modular inverse does not exist")
    return x % m

In [3]:
def matrix_mod_inv(matrix, m):
    """Compute the inverse of a matrix modulo m."""
    n = matrix.shape[0]
    # Calculate the determinant and its modular inverse.
    det = int(round(np.linalg.det(matrix)))
    det_inv = mod_inv(det % m, m)

    # Compute the adjugate (transpose of cofactor matrix)
    adj = np.zeros((n, n), dtype=int)
    for i in range(n):
        for j in range(n):
            # Create minor matrix for element (i,j)
            minor = np.delete(np.delete(matrix, i, axis=0), j, axis=1)
            cofactor = ((-1) ** (i + j)) * int(round(np.linalg.det(minor)))
            adj[j][i] = cofactor % m  # note the transpose here
    return (det_inv * adj) % m

In [4]:
def hill_encrypt(plaintext, key_matrix, m=26):
    """Encrypt plaintext using the Hill cipher."""
    plaintext = plaintext.upper().replace(" ", "")
    n = key_matrix.shape[0]
    # Pad plaintext if needed so that its length is a multiple of n.
    while len(plaintext) % n != 0:
        plaintext += "X"
    ciphertext = ""
    for i in range(0, len(plaintext), n):
        block = plaintext[i:i+n]
        vector = np.array([ord(c) - ord('A') for c in block]).reshape(n, 1)
        encrypted_vector = np.dot(key_matrix, vector) % m
        block_cipher = "".join(chr(int(num) + ord('A')) for num in encrypted_vector)
        ciphertext += block_cipher
    return ciphertext

In [ ]:
def hill_decrypt(ciphertext, key_matrix, m=26):
    """Decrypt ciphertext using the Hill cipher."""
    n = key_matrix.shape[0]
    inv_matrix = matrix_mod_inv(key_matrix, m)
    plaintext = ""
    for i in range(0, len(ciphertext), n):
        block = ciphertext[i:i+n]
        vector = np.array([ord(c) - ord('A') for c in block]).reshape(n, 1)
        decrypted_vector = np.dot(inv_matrix, vector) % m
        block_plain = "".join(chr(int(num) + ord('A')) for num in decrypted_vector)
        plaintext += block_plain
    return plaintext